## **Análisis de Sistemas Térmicos usando Python**

Clase 7:    Uso de biblioteca fluids para evaluar flujo en tuberías - **fsolve** 

La figura muestra un esquema de un sistema de dos estanques de agua conectados por dos cañerías con una bifurcación y una unión. 

Determine el caudal de estanque a estanque, cuando la válvula en la sección 2 está completamente cerrada.


![Ejemplo 3](M1_E3.png)

Datos:
* Cañerías de acero comercial
* Tramo 1: $L_1$ = 3 m, $D_1$ = 4"
* Tramo 2: $L_2$ = 90 m, $D_2$ = 3", 2 codos de 45º, 1 codo de 90º
* Tramo 3: $L_3$ = 150 m, , $D_3$ = 4", 1 codo de 90º
* Tramo 4: $L_4$ = 3 m, $D_4$ = 4"
* Uniones en T son despreciadas para el cálculo de pérdidas de carga.

In [5]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
from scipy import constants
from fluids.units import *
from scipy.optimize import fsolve

In [6]:
D_1 = 4 * constants.inch * u.m #Pulgadas a metros
D_2 = 3 * constants.inch * u.m  #Pulgadas a metros
D_3 = 4 * constants.inch * u.m  #Pulgadas a metros
D_4 = 4 * constants.inch * u.m  #Pulgadas a metros

L_1 = 3 * u.m   #metros
L_2 = 90 * u.m  #metros
L_3 = 150 * u.m #metros
L_4 = 3 * u.m 

z = 1 * u.m     #metros

material = 'Steel'
epsilon = fld.material_roughness(material)*u.m

fluid = "water"
T_o = 300 *u.K #Kelvin
rho = cp.PropsSI('D','T', T_o.magnitude, 'P',101325,fluid)*u.kg/u.m**3
mu = cp.PropsSI('V','T', T_o.magnitude, 'P',101325,fluid)*u.Pa*u.s

Si la válvula en la cañería 2 está cerrada, no hay flujo en 2

$V_2 = 0\\$
$\dot{V}_2 = 0$

La ecuación de Bernouilli se puede hacer por cada uno de los tramos: 1, 2, 3 y 4. Luego pueden sumarse entre sí. 
Haciendo Bernoulli por 1, 3 y 4, queda:

 $\rho g z_{AB}=\left( {\frac{1}{2}\rho V_{1}^{2}\left( {\frac{f_1 L_1}{D_1}+K_1} \right)} \right)+\frac{1}{2}\rho V_{3}^{2}\left( {\frac{f_3 L_3}{D_3}+K_3} \right) + \left( {\frac{1}{2}\rho V_{4}^{2}\left( {\frac{f_4 L_4}{D_4}+K_4} \right)} \right)$

 Como no hay flujo en 2 y como $D_1=D_3=D_4$, entonces $V_1 =V_3 =V_4$.
 Luego

  $\rho g z_{AB}= \frac{1}{2}\rho V_{1}^{2}\left(f_1 \frac{(L_1 + L_3 + L_4) }{D_1}+ \sum_i{K_i} \right)$

In [21]:
g = constants.g * u.m / u.s**2

K_elbow_90 = fld.fittings.bend_rounded_Crane(Di=D_1.magnitude, bend_diameters=1, angle=90)*u.dimensionless
K_globe = fld.fittings.K_gate_valve_Crane(D_1.magnitude, D_1.magnitude, 0)*u.dimensionless
K_exit = fld.exit_normal()*u.dimensionless
K_entrance = fld.entrance_sharp()*u.dimensionless

material = "Commercial steel"
epsilon = fld.material_roughness(material)*u.m

DP_g = rho * g * z

In [82]:
def flow_134(V_dot):
    V_dot = V_dot * u.m**3/u.s
    Re = (fld.Reynolds(D=D_1, rho=rho, V= (V_dot*4)/(np.pi*D_1**2), mu=mu)).to(u.dimensionless)
    f = fld.friction.friction_factor(Re, eD=epsilon/D_1, Method='Colebrook')
    return ((f * 0.5 * (L_1+L_3+L_4)/D_1 * rho * ((V_dot*4)/(np.pi*D_1**2))**2).magnitude + ((K_elbow_90+K_globe+K_exit+K_entrance)*((V_dot*4)/(np.pi*D_1**2))**2 * 0.5*rho).magnitude - DP_g.magnitude)

In [84]:
V_dot = 10/1000*u.m**3/u.s
V_dot_134 = fsolve(flow_134,V_dot)
V_dot_opt = V_dot_134*u.m**3/u.s
V_dot_opt.to(u.L/u.s)

Magnitude,[4.51885708551649]
Units,liter/second
